In [1]:
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

from jetbot.utils.utils import get_ip_address
import subprocess
import time

In [2]:
# OLED-SSD1306 초기화

# 하드웨어 I2C 방식의 128x32 Display
# gpio는 인식문제 해결을 위해 1로 설정
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None,i2c_bus=0,gpio=1)

# 디스플레이 초기화
disp.begin()

# 화면 클리어
disp.clear()
disp.display()

# 디스플레이 이미지 생성, 흑백(1-bit) 이미지 사용
disp_image = Image.new('1', (disp.width, disp.height))

# 이미지에 그릴 드로잉 객체 얻기
screen = ImageDraw.Draw(disp_image)

# 화면 전체를 검은색으로 채워서 지우기
screen.rectangle((0,0, disp.width, disp.height), outline=0,fill=0)

# 화면의 패딩영역 위치 조정 
screen_padding = -2
screen_top = screen_padding
screen_botton = disp.height - screen_padding
screen_x = 0

# 기본 서체 사용
font = ImageFont.load_default()

In [6]:
# OLED 화면에 기본 정보 디스플레이

### OLED 화면에 시스텐 정보 4줄을 보여주는 함수

## 로컬 IP 얻기
def getip():
    return str(get_ip_address('wlan0'))

def ShowInfo(strInfo):
    # 화면 전체를 검은색으로 채워서 지우기
    screen.rectangle((0,0, disp.width, disp.height), outline=0,fill=0)
    
    # 화면 맨 위의 첫 번째 줄에는 매개변수로 넘겨받을 문자열을 디스플레이
    screen.text((screen_x,screen_top+0), strInfo, font=font, fill=255)
    
    # 두 번째 줄에는 젯봇의 ip address를 디스플레이
    screen.text((screen_x,screen_top+8), "IP: " + getip(), font=font, fill=255)
    
    # 세 번째 줄에는 메모리 사용 상태를 디스플레이
    cmd = "free -m | awk 'NR=={printf \"Mem:%s/%sM %.2f%%\", $3,$2,$3*100/$2}'"
    MemUsage = subprocess.check_output(cmd,shell = True)
    screen.text((screen_x,screen_top+16), str(MemUsage.decode('utf-8')), font=font, fill=255)
    
    # 네 번째 줄에는 디스크 사용 상태를 디스플레이
    cmd = "df -h | awk 'NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3,$2,$5}'"
    Disk = subprocess.check_output(cmd,shell = True)
    screen.text((screen_x,screen_top+25), str(Disk.decode('utf-8')), font=font, fill=255)
    
    # 설정한 내용을 OLED 화면에 출력
    disp.image(disp_image)
    disp.display()

NameError: name 'GB' is not defined